<a href="https://colab.research.google.com/github/ankuj/teaching/blob/main/intro_nlp_day_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<br>
====================================================<br>
RNN Improvements Practical<br>
Vanishing/Exploding Gradients, GRUs and LSTMs<br>
====================================================<br>
Learning Goals:<br>
- Understand vanishing and exploding gradients in RNNs<br>
- Apply gradient clipping as a solution<br>
- Implement GRU and LSTM for text classification<br>
- Compare GRU vs LSTM on IMDB Sentiment Dataset<br>
====================================================<br>


In [2]:
!pip install torch
!pip install torchtext
!pip install transformers
!pip install datasets

  Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl (73.6 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
Using cached filelock-3.19.1-py3-none-any.whl (15 kB)
Using cached fsspec-2025.9.0-py3-none-any.whl (199 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Us

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

====================================================<br>
TASK 1 — Conceptual<br>
====================================================


<br>
Q1: Why do RNNs suffer from vanishing or exploding gradients?<br>
Write your answer here:<br>


In [ ]:
we suffer from RNN vanishing gradient problems because of long-term dependencies. This makes it difficult for the model to learn from distant time steps in the input sequence.


====================================================<br>
TASK 2 — Demonstrate Exploding Gradients <br>
====================================================

In [78]:

rnn = nn.RNN(input_size=1, hidden_size=1,nonlinearity='relu', batch_first=True)
x = torch.ones((1, 20, 1)) # long sequence
target = torch.tensor([1])  # fake label
criterion = nn.MSELoss()
optimizer = optim.SGD(rnn.parameters(), lr=1)

In [79]:
print("\n--- Task 2: Exploding Gradients Demonstration ---")
# Your code here
rnn.bias_hh_l0.data.fill_(0)
rnn.weight_hh_l0.data.fill_(-0.6052)   
rnn.weight_ih_l0.data.fill_(1.0)
for epoch in range(10):
    optimizer.zero_grad()
    output, _ = rnn(x)
    loss = criterion(output[:, -1, :], target.float().unsqueeze(1))
    loss.backward()
    optimizer.step()
    print(rnn.weight_hh_l0.grad.norm().item())
    print(_)
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


--- Task 2: Exploding Gradients Demonstration ---
0.10821466892957687
tensor([[[0.9042]]], grad_fn=<StackBackward0>)
Epoch 1, Loss: 0.009184992872178555
0.3373260796070099
tensor([[[1.2089]]], grad_fn=<StackBackward0>)
Epoch 2, Loss: 0.043619778007268906
0.6008071303367615
tensor([[[0.5160]]], grad_fn=<StackBackward0>)
Epoch 3, Loss: 0.23422054946422577
3.427645683288574
tensor([[[2.0375]]], grad_fn=<StackBackward0>)
Epoch 4, Loss: 1.0764743089675903
0.0
tensor([[[0.]]], grad_fn=<StackBackward0>)
Epoch 5, Loss: 1.0
0.0
tensor([[[0.]]], grad_fn=<StackBackward0>)
Epoch 6, Loss: 1.0
0.0
tensor([[[0.]]], grad_fn=<StackBackward0>)
Epoch 7, Loss: 1.0
0.0
tensor([[[0.]]], grad_fn=<StackBackward0>)
Epoch 8, Loss: 1.0
0.0
tensor([[[0.]]], grad_fn=<StackBackward0>)
Epoch 9, Loss: 1.0
0.0
tensor([[[0.]]], grad_fn=<StackBackward0>)
Epoch 10, Loss: 1.0



<br>
# --- TIP ---<br>
You should observe gradient norms growing very large, signifying an exploding gradient problem. Why is that the case? How can you remedy that?<br>


====================================================<br>
TASK 3 — Apply Gradient Clipping (15 mins)<br>
====================================================

In [81]:
print("\n--- Task 3: Gradient Clipping ---")
rnn = nn.RNN(input_size=1, hidden_size=1, batch_first=True)
optimizer = optim.SGD(rnn.parameters(), lr=1.0)
rnn.bias_hh_l0.data.fill_(0)
rnn.weight_hh_l0.data.fill_(-0.6052)
for epoch in range(5):
    optimizer.zero_grad()
    output, _ = rnn(x)
    loss = criterion(output[:, -1, :], target.float().unsqueeze(1))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=1.0)  # Gradient clipping
    optimizer.step()
    print(rnn.weight_hh_l0.grad)
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")



--- Task 3: Gradient Clipping ---
tensor([[0.2574]])
Epoch 1, Loss: 2.1356656551361084
tensor([[-0.2382]])
Epoch 2, Loss: 0.29114657640457153
tensor([[-0.0089]])
Epoch 3, Loss: 0.0025498804170638323
tensor([[-0.0078]])
Epoch 4, Loss: 0.0022285343147814274
tensor([[-0.0070]])
Epoch 5, Loss: 0.001977910753339529


In [ ]:
# Your code here

------------------------------<br>
Task 4: Manual Forward Pass <br>
------------------------------

In [ ]:
import numpy as np
def task4_manual_forward_pass():
    """
    Compute a forward pass manually (hidden and output state) for a small LSTM using the activation functions in the formula.
    Input sequence length T=3, input size=2, hidden size=2
    """
    x_seq = [np.array([0.5, -1.0]),
             np.array([1.0, 0.0]),
             np.array([-0.5, 0.5])]
    h_prev = np.zeros(2)
    c_prev = np.zeros(2)
    W_f = np.array([[0.1, 0.2], [0.3, 0.4]])
    U_f = np.array([[0.5, 0.6], [0.7, 0.8]])
    b_f = np.array([0.1, 0.1])
    W_i = np.array([[0.2, 0.1], [0.4, 0.3]])
    U_i = np.array([[0.6, 0.5], [0.8, 0.7]])
    b_i = np.array([0.1, 0.1])
    W_c = np.array([[0.3, 0.2], [0.1, 0.4]])
    U_c = np.array([[0.7, 0.6], [0.5, 0.8]])
    b_c = np.array([0.1, 0.1])
    W_o = np.array([[0.4, 0.3], [0.2, 0.1]])
    U_o = np.array([[0.8, 0.7], [0.6, 0.5]])
    b_o = np.array([0.1, 0.1]) 
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    for x in x_seq:
        f_t = sigmoid(np.dot(W_f, x) + np.dot(U_f, h_prev) + b_f)
        i_t = sigmoid(np.dot(W_i, x) + np.dot(U_i, h_prev) + b_i)
        c_tilde_t = np.tanh(np.dot(W_c, x) + np.dot(U_c, h_prev) + b_c)
        c_t = f_t * c_prev + i_t * c_tilde_t
        o_t = sigmoid(np.dot(W_o, x) + np.dot(U_o, h_prev) + b_o)
        h_t = o_t * np.tanh(c_t)
        h_prev, c_prev = h_t, c_t
    return h_t, c_t
     
task4_manual_forward_pass()
   

(array([0.10617523, 0.0964948 ]), array([0.19892013, 0.18283187]))

====================================================<br>
Preprocess IMDB dataset<br>
====================================================

In [85]:
print("\n--- Loading IMDB dataset ---")
oxford_restaurant_positive_reviews = [
    "Absolutely superb Sunday roast! The beef was tender, and the Yorkshire puddings were massive. Great atmosphere and friendly staff. Highly recommend for a classic meal.",
    "The vegan shepherd's pie here is a triumph! Rich, savory, and perfectly seasoned. Quick service, lovely decor. Best plant-based option in Oxford, hands down. A must-try.",
    "Incredible tapas selection. Every dish tasted fresh and authentic. The patatas bravas are a standout. Excellent wine list and a lively, buzzing ambiance. Will be back soon.",
    "Fantastic afternoon tea experience. Delicate sandwiches, warm scones, and exquisite pastries. Charming setting and attentive service. A perfect treat for a special occasion.",
    "Best pizza I've had outside of Italy. Thin crust, fresh ingredients, and baked to perfection in a wood-fired oven. Casual, fun spot. Great value for the quality. Five stars!",
    "The fish and chips were wonderfully crisp and not greasy at all. Generous portion size. Classic pub feel with a modern twist. A great place to unwind after a day of sightseeing.",
    "A hidden gem! Their homemade pasta is divine—simple but full of flavor. Intimate setting, perfect for a date night. Knowledgeable and welcoming staff. Definitely worth seeking out.",
    "Exceptional brunch menu! The eggs Benedict were flawless, and the coffee was strong and smooth. Bright, airy space with a relaxed vibe. Ideal spot for a weekend morning.",
    "The atmosphere is cozy and the cocktails are creative and delicious. Enjoyed their small plates thoroughly. Perfect spot for pre-theatre drinks and a quick bite. Loved the lighting.",
    "Simply the best curry in Oxford. Complex, aromatic flavors and incredibly tender lamb. The naan bread is freshly baked and huge. Quick takeaway service too. A regular favorite.",
    "The steak was cooked exactly medium-rare as requested—juicy and tender. Excellent quality meat and sides. Professional service and a sophisticated environment. Worth the higher price point.",
    "Charming little cafe with the most delicious homemade cakes. Great selection of teas. Friendly, welcoming staff, making it a perfect spot to relax and read. Felt very cozy.",
    "Fresh, vibrant sushi! Everything was beautifully presented and tasted so clean. Excellent vegetarian options available as well. Fast service. My new go-to for Japanese cuisine.",
    "Outstanding service and a delightful meal. The seasonal risotto was creamy and full of depth. Elegant dining room. Made our celebration feel truly special. Highly recommended for fine dining.",
    "Love the vibrant atmosphere and the amazing tacos! Fresh fillings and great homemade hot sauce. Margaritas are strong and tasty. Fun, energetic place for a group dinner. Great music.",
    "A quintessential Oxford pub experience! The real ale selection is top-notch, and the ploughman's lunch was hearty and fresh. Warm fire and great historical charm. True English comfort.",
    "Incredible value lunch special! The soup and sandwich combo was fresh and filling. Quick, efficient service—ideal for a working lunch. Clean and comfortable space. A reliable choice.",
    "The desserts here are works of art! We tried the chocolate lava cake, and it was rich and gooey. Great coffee to accompany it. Very attentive to dietary needs. Sweet end to the day.",
    "Best burgers in the city! Juicy patty, perfectly melted cheese, and a soft brioche bun. Excellent selection of craft beers to go with it. Casual, fun, and consistently great quality.",
    "The dim sum was excellent—steamed perfectly and full of flavor. Great variety on the menu. Polite, helpful staff. A fantastic place for a group feast. Very authentic feel. Will return.",
    "Fresh pasta made in-house daily. The Cacio e Pepe was simple perfection. Wonderful little Italian spot. Feels like a little slice of Rome in Oxford. Loved every single bite.",
    "Amazing courtyard seating for a sunny day! The salad with goat cheese and walnuts was incredibly refreshing. Great place for people-watching. Fast and friendly service. A real sun trap.",
    "The grilled seabass was cooked perfectly, flaky and moist. Excellent presentation. The service was impeccable—attentive but not intrusive. A genuinely delightful evening out. Top-tier dining.",
    "Cozy pub with a brilliant selection of local ales. The bar snacks were substantial and tasty. Great place to catch up with friends. Very friendly and laid-back atmosphere. Loved the beer garden.",
    "Sizzling fajitas that were flavourful and generous! Great homemade salsa and guacamole. Fun, colorful decor and a very upbeat vibe. Ideal for a celebratory dinner. Highly enjoyable.",
    "The coffee is expertly roasted and consistently excellent. Baristas are skilled and friendly. Best flat white in Oxford. Great spot for an early morning caffeine fix. Comfortable seating.",
    "Had the best steak sandwich I've ever tasted here. High-quality ingredients and perfectly seasoned. Quick service and reasonable prices. Great for a quick but satisfying meal.",
    "Truly authentic Thai cuisine. The Pad See Ew had the perfect smoky flavor. Generous portions. Staff are very warm and welcoming. Excellent value for such high-quality food. Spice level was spot on.",
    "Their homemade lemonade is wonderfully tangy and refreshing. Simple, clean food done really well. The avocado toast was excellent. Bright and cheerful cafe. Perfect for a light lunch.",
    "Excellent selection of cheeses and charcuterie. Great advice on wine pairings. Relaxed, sophisticated atmosphere. Perfect for a pre-dinner drink or a light European-style meal. A great find.",
    "The traditional English breakfast was spot-on—everything cooked perfectly. High-quality sausages and bacon. Friendly service and a welcoming environment. Set me up for the whole day. Terrific.",
    "Delicious vegetarian moussaka! Rich, comforting, and beautifully spiced. Great Greek wine selection. Warm, family-run feel to the restaurant. Highly recommend for a taste of the Mediterranean.",
    "Wonderful artisanal bread and pastries baked fresh daily. The sourdough is a revelation! Great for takeaway or a quick coffee and bite. Smells incredible inside. A morning essential.",
    "Perfect spot for a delicious cocktail and some small bites. The mixologists clearly know their stuff. Trendy but relaxed atmosphere. Great music playing in the background. A chic night out.",
    "The duck confit was sublime—crispy skin and tender meat. Excellent French cuisine in a beautiful setting. Attentive, formal service. Ideal for a refined dining experience. Pure class.",
    "Fantastic value, all-you-can-eat buffet that doesn't compromise on quality. Huge variety of fresh, tasty dishes. Staff are efficient and keep everything topped up. Great for a large appetite.",
    "The seafood risotto was packed with fresh mussels, prawns, and squid. Clearly high-quality ingredients. Excellent presentation and prompt service. Felt like a coastal holiday meal.",
    "Great place for hearty, classic pub grub. The pie of the day was substantial and flavorful. Cozy corners and friendly locals. The perfect place to hide from the rain. Very welcoming.",
    "Brilliant coffee shop near the colleges. Great vibe for studying or meeting friends. Their pastries are always fresh. Reliable Wi-Fi. A comfortable and productive space to hang out.",
    "The pulled pork sandwich was juicy and smoky, served with excellent coleslaw. Informal, fun atmosphere. Great selection of craft beers and ciders. A solid choice for American BBQ-style food.",
    "Such a charming and quiet tea room. The scones were incredibly light and fluffy. A true taste of traditional England. Polite, professional staff. Highly recommend for a relaxing break.",
    "Loved the spicy chicken wings! Excellent appetizer and generous portions. Great selection of beers on tap. Friendly staff and fast service. Perfect for watching a match. Lively spot.",
    "The homemade soup changes daily and is always exceptional. Fresh, wholesome ingredients. Great for a quick, healthy lunch. Cozy environment and reasonable prices. A reliable choice.",
    "Excellent ramen! Rich, savory broth and perfectly cooked noodles. Great toppings and very quick to be served. Ideal comfort food on a cold day. Authentic Japanese flavors. Delicious.",
    "Outstanding service and delicious food. The pan-fried cod was light and flavorful. Elegant dining without feeling stuffy. Very impressed with the wine list recommendations. Great night.",
    "Best breakfast burrito I’ve ever had! Stuffed with flavour and just the right amount of spice. Great coffee too. Friendly, upbeat staff. Perfect way to start a busy day. Highly recommend.",
    "A cozy, old-school pub with a roaring fire and great atmosphere. Excellent selection of local ales. The bar snacks were top-notch. Very friendly service. A perfect winter retreat.",
    "Fantastic wood-fired pizza! The crust was chewy and charred perfectly. High-quality toppings. Quick service and a bustling, cheerful vibe. Definitely a family favorite spot.",
    "The vegan burger here is genuinely incredible—firm, flavorful, and satisfying. Great fries on the side. Casual and modern decor. So many great plant-based options. Excellent spot.",
    "Beautiful garden area for outdoor dining. The summer salad was fresh and delicious. Attentive and quick service. Perfect spot for a sunny lunch. A real urban escape. Very relaxing.",
    "Wonderful little Italian place. The homemade ravioli was divine and the sauce was excellent. Intimate, warm setting. Very attentive service. Will definitely be returning soon.",
    "Absolutely superb steak and kidney pie. Rich gravy and flaky pastry. A true taste of classic pub food. Great selection of beers to accompany it. Cozy and welcoming. Perfect meal.",
    "The coffee is exceptional and the pastries are baked fresh every morning. Great spot to read and relax. Friendly baristas and a bright, airy space. My favorite morning ritual spot.",
    "Had the best Eggs Florentine! Perfectly poached eggs and creamy hollandaise sauce. Excellent brunch spot. Quick service and reasonable prices. Ideal for a weekend treat. Loved it.",
    "Incredible Indian food. The butter chicken was rich, creamy, and spiced just right. Fresh naan bread. Great atmosphere and attentive service. A consistent favorite in Oxford. Must try.",
    "The fish tacos were fresh, zesty, and perfectly seasoned. Great casual dining atmosphere. Fun, vibrant decor. Strong, tasty margaritas. A great place for a fun night out with friends.",
    "Charming traditional tea room. The sandwiches and cakes were delightful. Attentive service. Perfect for a special occasion or a quiet afternoon treat. Very relaxing and elegant.",
    "A hidden gem! Their homemade pasta is superb—simple but full of flavor. Intimate, cozy setting. Wonderful wine list. Knowledgeable and welcoming staff. A truly lovely dining experience.",
    "Fantastic value for money at lunchtime. The soup and sandwich were fresh and filling. Very quick and efficient service. Clean and comfortable space. A great go-to for a quick bite.",
    "The cocktails are creative and delicious. The atmosphere is cozy and chic. Enjoyed the small plates thoroughly. Perfect for pre-dinner drinks and a snack. Great date night spot.",
    "Best pizza in the area! Thin crust, fresh toppings, baked to perfection. Casual, fun spot. Excellent value for the high quality. Friendly staff. Consistently great food.",
    "Exceptional service and delightful meal. The seasonal risotto was creamy and flavorful. Elegant dining room. Made our celebration feel truly special. Highly recommended for a treat.",
    "Love the energetic atmosphere and the amazing tacos! Fresh fillings and great homemade hot sauce. Margaritas are strong. Fun, energetic place for a group dinner. Great service.",
    "A quintessential Oxford pub experience! The real ale selection is top-notch, and the ploughman's lunch was hearty. Warm fire and historical charm. True English comfort food here.",
    "The desserts here are incredible! We tried the sticky toffee pudding—it was rich and delicious. Great coffee to accompany it. Very attentive to dietary needs. Sweet and perfect.",
    "Best burgers in the city! Juicy patty, perfectly melted cheese, and a soft bun. Excellent selection of craft beers. Casual, fun, and consistently great quality. Highly recommend.",
    "The dim sum was excellent—steamed perfectly and full of flavor. Great variety on the menu. Polite, helpful staff. A fantastic place for a group feast. Very authentic. Will return soon.",
    "Fresh pasta made in-house daily. The simplicity of the dishes lets the fresh ingredients shine. Wonderful little Italian spot. Feels authentic. Loved every single bite. Perfect.",
    "Amazing courtyard seating for a sunny day! The halloumi salad was refreshing and delicious. Great place for people-watching. Fast and friendly service. A real sun trap. Relaxing.",
    "The grilled salmon was cooked perfectly, flaky and moist. Excellent presentation. The service was impeccable—attentive but not intrusive. A genuinely delightful evening out. Top dining.",
    "Cozy pub with a brilliant selection of local ales. The bar snacks were substantial and tasty. Great place to catch up with friends. Very friendly and laid-back atmosphere. Enjoyed it.",
    "Sizzling fajitas that were flavourful and generous! Great homemade salsa and guacamole. Fun, colorful decor and a very upbeat vibe. Ideal for a celebratory dinner. Highly enjoyable.",
    "The coffee is expertly roasted and consistently excellent. Baristas are skilled and friendly. Best flat white in Oxford. Great spot for an early morning fix. Comfortable seating.",
    "Had the best chicken sandwich I've ever tasted here. High-quality ingredients and perfectly seasoned. Quick service and reasonable prices. Great for a satisfying quick meal. Loved it.",
    "Truly authentic Thai cuisine. The green curry had the perfect balance of spice and creaminess. Generous portions. Staff are very warm and welcoming. Excellent value for money. Spot on.",
    "Their homemade lemonade is wonderfully tangy and refreshing. Simple, clean food done really well. The poached eggs on toast were excellent. Bright and cheerful cafe. Perfect for lunch.",
    "Excellent selection of cheeses and charcuterie. Great advice on wine pairings. Relaxed, sophisticated atmosphere. Perfect for a pre-dinner drink or a light meal. A great find in Oxford.",
    "The traditional English breakfast was spot-on—everything cooked perfectly. High-quality ingredients. Friendly service and a welcoming environment. Set me up for the whole day. Terrific.",
    "Delicious vegetarian lasagne! Rich, comforting, and beautifully layered. Great house wine selection. Warm, family-run feel. Highly recommend for a taste of comforting Italian food.",
    "Wonderful artisanal bread and pastries baked fresh daily. The croissants are heavenly! Great for takeaway or a quick coffee and bite. Smells incredible inside. A morning must-have.",
    "Perfect spot for a delicious cocktail and some small bites. The mixologists clearly know their craft. Trendy but relaxed atmosphere. Great music. A chic night out in the city.",
    "The beef bourguignon was sublime—tender meat and rich, savory sauce. Excellent French cuisine in a beautiful setting. Attentive service. Ideal for a refined dining experience. Pure class.",
    "Fantastic value, three-course set menu that doesn't compromise on quality. Huge variety of fresh, tasty dishes. Staff are efficient and helpful. Great for a large appetite. Highly recommend.",
    "The seafood linguine was packed with fresh seafood. Clearly high-quality ingredients. Excellent presentation and prompt service. Felt like a coastal holiday meal. Very impressive.",
    "Great place for hearty, classic pub grub. The bangers and mash were substantial and flavorful. Cozy corners and friendly atmosphere. The perfect place to unwind. Very welcoming pub.",
    "Brilliant coffee shop near the colleges. Great vibe for studying or meeting friends. Their cakes are always fresh. Reliable Wi-Fi. A comfortable and productive space to hang out. Love it.",
    "The pulled jackfruit sandwich was juicy and smoky, served with excellent slaw. Informal, fun atmosphere. Great selection of craft beers. A solid choice for vegetarian BBQ-style food.",
    "Such a charming and quiet tea room. The fruit scones were incredibly light and fluffy. A true taste of traditional England. Polite, professional staff. Highly recommend for a break.",
    "Loved the spicy cauliflower wings! Excellent appetizer and generous portions. Great selection of beers on tap. Friendly staff and fast service. Perfect for a casual evening out. Lively.",
    "The homemade soup changes daily and is always exceptional. Fresh, wholesome ingredients. Great for a quick, healthy lunch. Cozy environment and reasonable prices. A reliable choice.",
    "Excellent ramen! Rich, savory broth and perfectly cooked noodles. Great toppings and very quick to be served. Ideal comfort food on a cold day. Authentic Japanese flavors. Delicious.",
    "Outstanding service and delicious food. The pan-fried halibut was light and flavorful. Elegant dining without feeling stuffy. Very impressed with the wine list. A great night out.",
    "Best breakfast toastie I’ve ever had! Stuffed with flavour and just the right amount of seasoning. Great coffee too. Friendly, upbeat staff. Perfect way to start a busy day. Highly recommend.",
    "A cozy, old-school pub with a beautiful beer garden. Excellent selection of local ciders. The bar snacks were substantial and tasty. Very friendly service. A perfect summer spot.",
    "Fantastic thin-crust pizza! The base was chewy and charred perfectly. High-quality toppings. Quick service and a bustling, cheerful vibe. Definitely a family favorite spot.",
    "The vegetarian moussaka is genuinely incredible—firm, flavorful, and satisfying. Great side salad. Casual and modern decor. So many great plant-based options. Excellent spot for veggies.",
    "Beautiful outdoor seating area. The beetroot and feta salad was incredibly refreshing. Attentive and quick service. Perfect spot for a sunny lunch. A real urban escape. Very relaxing.",
    "Wonderful little Italian spot. The gnocchi was light and the pesto sauce was excellent. Intimate, warm setting. Very attentive service. Will definitely be returning soon. Authentic.",
    "Absolutely superb cottage pie. Rich gravy and fluffy potato topping. A true taste of classic pub food. Great selection of beers to accompany it. Cozy and welcoming. Perfect meal.",
    "The coffee is exceptional and the pastries are baked fresh every morning. Great spot to read and relax. Friendly baristas and a bright, airy space. My favorite morning ritual spot.",
    "Had the best Eggs Royal! Perfectly poached eggs and creamy hollandaise sauce. Excellent brunch spot. Quick service and reasonable prices. Ideal for a weekend treat. Loved the atmosphere.",
    "Incredible Indian food. The lamb rogan josh was tender and spiced just right. Fresh naan bread. Great atmosphere and attentive service. A consistent favorite in Oxford. Must try.",
    "The halloumi fries were fresh, zesty, and perfectly seasoned. Great casual dining atmosphere. Fun, vibrant decor. Strong, tasty cocktails. A great place for a fun night out with friends.",
    "Charming traditional tea room. The sandwiches and cakes were delightful. Attentive service. Perfect for a special occasion or a quiet afternoon treat. Very relaxing and elegant."
]
oxford_restaurant_negative_reviews = [
    "The wait time was excessive, nearly an hour for a simple burger. The food arrived cold and overcooked. Service was absent-minded and unapologetic. Very disappointing experience.",
    "Expensive and poor quality. The pasta was bland and mushy, clearly pre-made. Small portions and a very rushed feeling. Definitely not worth the high price tag. Avoid this place.",
    "My reservation was lost, and we waited 30 minutes for a table. The service was slow and indifferent. The steak was gristly and tough. Left feeling ripped off and frustrated.",
    "The atmosphere was loud and chaotic, impossible to have a conversation. Our table was sticky and the floor was dirty. The food was mediocre at best. Will not be returning.",
    "Ordered the fish and chips; the fish was greasy, and the batter was soggy. Staff seemed completely overwhelmed and stressed. A truly substandard pub meal. Huge letdown.",
    "The vegetarian option was tiny and tasted like unseasoned vegetables. Zero creativity or flavor. Slow to take our order and even slower to deliver the bill. Bad value.",
    "Cocktails were weak and overpriced, tasting mostly of cheap mixer. The bar area was cramped and poorly lit. Music was too loud and aggressive. A terrible place for a drink.",
    "Incredibly rude waiter who clearly didn't want to be there. The coffee was lukewarm and bitter. The cake was stale and dry. Left quickly, wishing I hadn't bothered.",
    "The curry was watery and lacked any real depth of spice or flavour. The naan was burnt around the edges. Took forever for the food to arrive. Much better Indian places nearby.",
    "The 'signature' dish was incredibly salty, almost inedible. Sent it back, but the replacement was just as bad. No apology or compensation offered. Terrible customer care.",
    "Extremely cramped seating; we were practically on the next table's lap. The air conditioning wasn't working, making it uncomfortably warm. Food was forgettable. Unpleasant visit.",
    "The burger was dry and the bun was soggy—a clear sign it had been sitting under a heat lamp. Fries were cold. Basic food done very badly. Wouldn't recommend this place.",
    "Our starter never arrived, but we were still charged for it. Had to argue to get it removed from the bill. The main course was equally unimpressive. Poor organization.",
    "Bad hygiene standards. Saw a hair in my food and immediately lost my appetite. Staff didn't seem surprised or concerned. Left immediately and won't be coming back.",
    "The roast dinner was served cold and the potatoes were rock-hard. Gravy tasted instant and metallic. Clearly reheated food. They've dropped the ball big time here.",
    "Very limited menu with boring, uninspired choices. Food took ages, and when it arrived, it was lukewarm. Overpriced for what it is. Look elsewhere for dinner.",
    "The pizza base was thin, dry, and cracker-like. Toppings were scarce and low-quality. Felt like a frozen pizza. Disappointing and not worth the money. Go to a chain instead.",
    "Rude hostess who acted like we were an inconvenience for showing up with a booking. Sat us in a terrible spot. Service was slow throughout. A truly unwelcoming place.",
    "The atmosphere felt sterile and lacked any character. The music was distracting elevator jazz. Food was completely forgettable and overpriced. No warmth here at all.",
    "Tried the breakfast and it was a disaster. Poached eggs were solid, bacon was greasy. Waited 45 minutes for it to be served. Complete waste of a morning. Avoid brunch.",
    "Terrible experience. The soup was cold and gloopy. The bread was dry. Waitress was impatient and rushed us. Felt like they wanted us in and out instantly. Never again.",
    "The seating was very uncomfortable—hard wooden benches with no cushions. Food was expensive and the quality was low. Left with a sore back and an empty wallet. Not enjoyable.",
    "Ordered a rare steak, got a well-done, dried-out piece of meat. Waiter argued when I pointed it out. They did not correct the order. Completely ruined the evening.",
    "The restaurant smells strongly of stale fryer oil. The food tasted exactly like it smelled—heavy and greasy. Had indigestion immediately after leaving. Unpleasant experience.",
    "Clearly understaffed. Waited 20 minutes just for water and menus. Food was completely average and boring when it finally arrived. Service needs serious improvement.",
    "The vegan dish was extremely bland and poorly presented. Felt like an afterthought thrown on a plate. Zero effort put into the plant-based options. Not acceptable.",
    "Portion sizes were ridiculously small for the price. Left feeling hungry and annoyed. The wine list was also hugely overpriced. Will not be back for dinner.",
    "The table next to us was loud and disruptive, and the staff did nothing to address it. Ruined the ambiance. Food was just okay, nothing special. Poor atmosphere control.",
    "The coffee was burnt and bitter. The milk tasted off. Didn't even finish my drink. Baristas seemed more interested in talking to each other. Terrible coffee shop.",
    "The outdoor seating area was dirty and neglected, covered in litter. Food was average and took a long time. Definitely needs a proper clean-up. Not a pleasant setting.",
    "Poor value for money. Paid a fortune for small, tasteless tapas dishes. Felt like we were eating frozen reheated food. Much better places to spend your money in Oxford.",
    "The waiter spilled a drink all over our table and barely apologized, just wiped it quickly. Service was careless and unprofessional. The main course was lukewarm.",
    "The restaurant was incredibly chilly inside, felt like eating in a fridge. Had to keep my coat on. Food got cold quickly. Uncomfortable and rushed meal. Poor heating.",
    "The chicken in my dish was dry and stringy, clearly overcooked. The sauce was overly sweet and artificial tasting. Very poor execution of a simple meal. Disappointing.",
    "We were seated right next to the noisy service station. Constant clattering of cutlery. Ruined the dining experience. Staff ignored requests to move. Bad table planning.",
    "The soup was watery and lacked seasoning. The bread was stale. Extremely basic food for a high price. Felt like a rip-off. They need to up their game considerably.",
    "Terrible customer service. Waited 45 minutes for food, then they brought the wrong order. No sincere apology. Had to wait even longer for the correct dish. Avoid.",
    "The restroom facilities were disgusting and unkempt. A clear sign of poor hygiene standards overall. Lost my appetite after seeing them. Unacceptable for a city restaurant.",
    "The whole place felt dingy and dated. Clearly needs a refurbishment and a deep clean. Food was uninspired and overpriced. A very depressing dining atmosphere.",
    "Tried the pie of the day; the crust was burnt and the filling was sparse. Tasted like it came from a cheap supermarket. Zero care put into the cooking. Avoid the pies.",
    "Overly aggressive music playing, made it hard to relax or talk. Food was average, nothing to justify the price. Left with a headache. Needs to turn the volume down.",
    "The salad greens were wilted and brown around the edges. Clearly not fresh ingredients. Sent it back. Staff didn't seem bothered. Poor attention to detail. Bad lunch.",
    "Expensive drinks and incredibly slow bar staff. Took 15 minutes to get a simple glass of wine. Crowded and disorganized bar area. Not a place for a quick drink.",
    "Had the pancakes—they were heavy, doughy, and undercooked in the middle. The topping was minimal. Very poor brunch execution. Definitely not worth the wait or the price.",
    "The smell of chlorine was overpowering, clearly trying to mask something else. Food was bland and forgettable. Unpleasant environment. Will not return to this place.",
    "The chips were soggy and heavily salted, almost inedible. The main course was equally disappointing—dry and tasteless. Very poor quality control from the kitchen.",
    "We felt completely rushed by the server from start to finish. Felt pressured to order quickly and vacate the table. Ruined the leisurely dinner. Poor hosting manners.",
    "The wine list was limited and incredibly overpriced for the quality offered. Food was acceptable but not good enough to justify the high bill. Disappointing value.",
    "The whole experience felt corporate and soulless. No charm or character whatsoever. Food was mass-produced and uninspired. A generic and forgettable restaurant.",
    "The soup was lukewarm and had a strange, artificial taste. Sent it back but the replacement wasn't much better. Service was grudging. A very poor start to the meal.",
    "Extremely loud, abrasive noises from the kitchen area. Ruined the ability to hold a conversation. Food was passable but the noise made it unpleasant. Needs better soundproofing.",
    "The waiter had zero knowledge of the wine list or the menu. Had to keep running back to ask questions. Service was incompetent and slow. Very frustrating experience.",
    "The only vegan main course was sold out by 7 PM. Very poor preparation for a Saturday night. Ended up with a mediocre side dish. Disappointing lack of options.",
    "The cutlery and glassware were visibly dirty and smeared. Had to ask for replacements. Basic hygiene failing. Lost confidence in the kitchen. Very unpleasant start.",
    "The chicken burger was mostly dry breading and gristle. Very low-quality meat used. Tasted cheap and processed. Avoid the poultry options here. Not good at all.",
    "Atmosphere was depressing and quiet, like a library. Zero energy in the room. Food was boring and overpriced. Not a place for a fun evening out. Needs a vibe lift.",
    "They added an automatic service charge without asking, despite the poor service we received. Very cheeky and annoying. The meal was not worth the price. Will dispute the charge.",
    "The vegetarian curry was mostly sauce with very few vegetables. Felt like a rip-off portion-wise. High price for a watery dish. Very disappointing offering.",
    "The salad was tiny and drowned in a strong, vinegary dressing. Could barely taste the fresh ingredients. Overdressed and under-portioned. Bad value for a salad.",
    "The coffee machine was broken, so no hot drinks were available. Waited 15 minutes for them to tell us. Very poor communication and organization. Wasted time here.",
    "My food arrived 10 minutes after my partner's, leaving me eating alone. Poor coordination from the kitchen staff. No apology for the delay. Unprofessional service.",
    "The outdoor area was full of smokers, which ruined the meal for non-smokers. Staff did nothing to manage the area. Unpleasant environment. Won't sit outside again.",
    "Tried the special and it was incredibly disappointing—dry fish and undercooked rice. Clearly a rushed dish. The service was equally mediocre. Big letdown special.",
    "The waiter kept trying to upsell us on every single thing, making us feel pressured. Very pushy and annoying. Ruined the relaxed feel. Just give us the menu.",
    "The chips were cold, and the main dish was lukewarm. Sent them back, and they returned 10 minutes later slightly warmer. Clearly just microwaved. Unacceptable standard.",
    "The whole restaurant smelled strongly of industrial cleaning products. Lost my appetite almost immediately. The food tasted bland and chemical. Not a good dining experience.",
    "The music was cheesy pop covers and repetitive, making for a terrible background atmosphere. Food was acceptable but the music was a definite mood killer. Needs new playlist.",
    "The tea was weak and tasted of old kettle limescale. The milk was warm. Poor quality control on simple drinks. Avoid hot beverages here. Terrible tea and coffee.",
    "The table was dirty when we sat down, covered in crumbs from the previous guests. Had to ask for it to be wiped. Poor attention to basic hygiene. Off-putting start.",
    "My partner's starter arrived 15 minutes after mine. Poor timing management. The main course was equally uncoordinated. Amateur service and poor kitchen flow.",
    "The bread was stale and hard, clearly not fresh that day. A small detail but speaks volumes about the quality control. The main course was average. Disappointing bread.",
    "The children's menu was overpriced and the quality was terrible—cheap, greasy nuggets. No healthy options available. Not family-friendly despite appearances. Avoid kids' food.",
    "We were completely ignored by the staff for the first 20 minutes. Had to flag down someone just for water. Service was non-existent. Very frustrating wait time.",
    "The restaurant felt like a poorly converted office space—no charm or atmosphere. Food was boring and generic. Wouldn't recommend this soulless place. Zero character.",
    "The chicken was undercooked in the centre and pink. Immediately sent it back. Had to wait ages for a replacement. A serious health and safety concern. Be cautious here.",
    "The dessert was clearly defrosted and still partly frozen in the middle. Cheap, pre-made quality. Very poor end to the meal. Avoid the desserts entirely. Not homemade.",
    "The chips were drenched in oil, pooling on the plate. Heavy and greasy, almost inedible. Main course was dry. The entire meal felt unhealthy and poorly prepared.",
    "Waited 15 minutes for the bill and then another 10 minutes for the card machine. Staff seemed to prioritize everything else. Extremely slow checkout process. Annoying wait.",
    "The whole place felt cold and unwelcoming. The server barely smiled or made eye contact. Zero personality or hospitality. Just going through the motions. Poor service.",
    "The portion sizes were inconsistent across the table. One person got a huge meal, the other tiny. Unfair and disorganized. Food quality was mediocre at best. Avoid.",
    "The atmosphere was ruined by a loud group of drunk patrons who were not managed by staff. Made for a very unpleasant dining experience. Staff were passive. Poor management.",
    "The coffee was weak and watery. Clearly trying to save money on the beans. Tasted like dishwater. Very poor quality beverage. Don't order a black coffee here.",
    "The table was wobbly and kept rocking, spilling my drink slightly. Staff didn't offer to fix it. Minor detail, major annoyance. Poor attention to basic setup. Frustrating.",
    "The menu descriptions were misleading and the food received bore little resemblance to the listing. Disappointed by the actual product. Over-promised, under-delivered.",
    "The heating was blasting, making the room uncomfortably hot and stuffy. Food got cold quickly due to the heat. Very unpleasant environment for dining. Poor temperature control.",
    "The waiter kept clearing plates before everyone at the table was finished eating. Felt rushed and intrusive. Poor table service etiquette. Rushed us out the door.",
    "The supposed fresh juice tasted like cheap, concentrated syrup. Clearly not fresh as advertised. Felt misled and ripped off. Avoid the 'fresh' drinks here.",
    "The food was bland and completely under-seasoned. Had to heavily salt and pepper everything. Chef needs to taste the food before sending it out. Zero flavor profile.",
    "The entire meal took over two hours due to slow service and kitchen delays. Far too long for a casual dinner. Ruined the evening plans. Poor timing management.",
    "The children's high chair provided was visibly sticky and dirty. Had to clean it myself. Poor hygiene standards for families. Unacceptable level of cleanliness. Avoid.",
    "The wine we ordered was corked and tasted awful. The waiter argued it was 'fine' before finally replacing it. Unprofessional and poor product knowledge. Bad wine experience.",
    "The chips were cold and the main course was undercooked. Sent it back, and the replacement took forever. A complete disaster of a meal. Very poor kitchen execution.",
    "The whole place felt overpriced for the basic, bland quality of the food. Paying for the location, not the culinary skill. Disappointing value. Look elsewhere.",
    "The noise level from the music and chatter was so high it felt physically stressful. Impossible to relax. Food was mediocre. Avoid if you want a peaceful meal. Too loud.",
    "The salad had clearly been pre-made hours ago; the dressing made the leaves soggy. Tasted tired and old. Poor food preparation standards. Not fresh at all.",
    "The restaurant was incredibly dark and poorly lit, making it hard to read the menu and see the food. Unpleasant atmosphere. Felt a bit dingy. Too dark for dinner.",
    "The waiter had a very strong body odor, which was off-putting throughout the meal. Unprofessional and unpleasant. Service was also slow. Hygiene issue.",
    "The only tap water available was warm. Had to ask three times for ice. Poor attention to basic customer needs. Food was equally uninspired. Poor attention.",
    "The main dish was tiny, more like an appetizer size. Felt cheated on the portion size for the price. Left hungry and dissatisfied. Poor value for money here.",
    "The vegetarian burger was dense and rubbery, tasted like frozen processed food. Clearly not homemade. Poor quality patty. Avoid the veggie options here.",
    "The service charge was added despite the slow and unhelpful waiter. Felt like a compulsory tax for poor service. Disappointed by the aggressive charging policy."
]



--- Loading IMDB dataset ---


In [ ]:
oxford_reviews_labels = {
    "positive": oxford_restaurant_positive_reviews,
    "negative": oxford_restaurant_negative_reviews
}
import string
from torch.utils.data import Dataset, DataLoader

class ReviewDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=50):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.max_len = max_len

    def encode_text(self, text):
        tokens = text.lower().split()
        ids = [self.vocab.get(t, self.vocab["<UNK>"]) for t in tokens]

        if len(ids) < self.max_len:
            ids += [self.vocab["<PAD>"]] * (self.max_len - len(ids))
        else:
            ids = ids[:self.max_len]

        return torch.tensor(ids)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.encode_text(self.texts[idx]), torch.tensor(self.labels[idx])
    
words=[]
for review in oxford_restaurant_positive_reviews:
    print(review)
    for word in review.split():
        words+=word.strip(string.punctuation)
        
        


    

Absolutely superb Sunday roast! The beef was tender, and the Yorkshire puddings were massive. Great atmosphere and friendly staff. Highly recommend for a classic meal.
Absolutely
superb
Sunday
roast
The
beef
was
tender
and
the
Yorkshire
puddings
were
massive
Great
atmosphere
and
friendly
staff
Highly
recommend
for
a
classic
meal
The vegan shepherd's pie here is a triumph! Rich, savory, and perfectly seasoned. Quick service, lovely decor. Best plant-based option in Oxford, hands down. A must-try.
The
vegan
shepherd's
pie
here
is
a
triumph
Rich
savory
and
perfectly
seasoned
Quick
service
lovely
decor
Best
plant-based
option
in
Oxford
hands
down
A
must-try
Incredible tapas selection. Every dish tasted fresh and authentic. The patatas bravas are a standout. Excellent wine list and a lively, buzzing ambiance. Will be back soon.
Incredible
tapas
selection
Every
dish
tasted
fresh
and
authentic
The
patatas
bravas
are
a
standout
Excellent
wine
list
and
a
lively
buzzing
ambiance
Will
be
back
soo

====================================================<br>
TASK 5 — Implement LSTM Sentiment Classifier on the IMDB dataset<br>
====================================================

In [ ]:
class GRUClassifier(nn.Module):

    
    

====================================================<br>
TASK 6 — Swap LSTM with GRU and repeat Task 4<br>
====================================================

====================================================<br>
TASK 7 <br>
====================================================

Compare loss curves for the LSTM and GRU classifiers. Which performs better and why?<br>